# Table of Contents
 <p>

In [1]:
using DataFrames
using GaussianProcesses
using Proj4
using GaussianProcesses: set_params!
;

In [2]:
using JLD

In [3]:
include("src/preprocessing.jl")
include("src/variogram.jl")

cov (generic function with 1 method)

In [4]:
isdList=read_isdList()
isdSubset=isdList[[(usaf in (725450,725460,725480,725485)) for usaf in isdList[:USAF].values],:]
isdSubset

,USAF,WBAN,NAME,CTRY,STATE,ICAO,LAT,LON,ELEV,BEGIN,END,X_PRJ,Y_PRJ
1,725450,14990,THE EASTERN IOWA AIRPORT,US,IA,KCID,41.883,-91.717,264.6,1973,2015,1.64799e6,1.0441e6
2,725460,14933,DES MOINES INTERNATIONAL AIRPORT,US,IA,KDSM,41.534,-93.653,291.7,1973,2015,1.48723e6,1.00379e6
3,725480,94910,WATERLOO MUNICIPAL AIRPORT,US,IA,KALO,42.554,-92.401,264.6,1960,2015,1.59025e6,1.11766e6
4,725485,14940,MASON CITY MUNICIPAL ARPT,US,IA,KMCW,43.154,-93.327,373.4,1973,2015,1.51407e6,1.18374e6


In [5]:
hourly_cat=read_Stations(isdSubset)
itest=3

3

In [6]:
include("src/fitted_kernel.jl")

Type: GaussianProcesses.ProdKernel
  Type: GaussianProcesses.Masked{GaussianProcesses.SumKernel}, Params: [-0.0806483,1.0449,1.50786,1.10795,-1.38548,-1.22736,-1.05138,3.09723,1.28737,2.84127,3.64666,0.469691,3.00962,7.70695,-5.39838]
  Type: GaussianProcesses.Masked{GaussianProcesses.SEIso}, Params: [11.2445,0.132749]


In [7]:
module pred
    include("src/predict_from_nearby.jl")
end

pred

In [8]:
test_usaf=get(isdSubset[itest,:USAF])

725480

In [20]:
dt_start=DateTime(2015,1,1,0,0,0)
increm=get(maximum(hourly_cat[:ts])-minimum(hourly_cat[:ts])) / 15
window=3*increm

6307128000 milliseconds

In [21]:
while true
    dt_end=dt_start+window
    nearby_pred = pred.predict_from_nearby(hourly_cat, isdSubset, 
        k_spatiotemporal, logNoise,
        itest, dt_start, dt_end)
    save(@sprintf("saved/predictions_from_nearby/%d_%s_to_%s.jld", test_usaf, Date(dt_start), Date(dt_end)), 
        "nearby_pred", 
        nearby_pred)
    if dt_end >= get(maximum(hourly_cat[:ts]))
        break
    end
    dt_start+=increm
end